In [29]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing.sequence import pad_sequences

In [28]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [26]:
id_train_features = '1LnKdehYxXYXIUhxO1NFeu5GebmKpbYjY'
id_test_features = '1CIuprbEUeJDEdg0WO5Sq2-xV8OBElXLX'
id_train_labels = '1dzgH430IP8VV4Jw6InbfTiMEFUxZk1pO'
id_test_labels = '10dwdJ5EI_w-HfeM3F8CyctlqAnuW3RY9'

In [30]:
downloaded = drive.CreateFile({'id':id_train_features})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_features.csv')  
# Dataset is now stored in a Pandas Dataframe
train_X = pd.read_csv('train_features.csv',header=None)
train_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.087,0.106,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072
1,0.106,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228
2,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123
3,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123,0.019
4,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123,0.019,0.123


In [5]:
downloaded = drive.CreateFile({'id':id_test_features})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_features.csv')  
# Dataset is now stored in a Pandas Dataframe
test_X = pd.read_csv('test_features.csv',header=None)
test_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096,0.236
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096,0.236,0.082


In [6]:
downloaded = drive.CreateFile({'id':id_train_labels})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_labels.csv')  
# Dataset is now stored in a Pandas Dataframe
train_Y = pd.read_csv('train_labels.csv',header=None)
train_Y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [7]:
downloaded = drive.CreateFile({'id':id_test_labels})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_labels.csv')  
# Dataset is now stored in a Pandas Dataframe
test_Y = pd.read_csv('test_labels.csv',header=None)
test_Y.head()

,0,1,2,3,4,5
0,c,l,a,s,s,2
1,c,l,a,s,s,2
2,c,l,a,s,s,2
3,c,l,a,s,s,2
4,c,l,a,s,s,2


In [8]:
test_Y = test_Y[5]

In [56]:
row = np.array(train_X.iloc[1])
np.concatenate((row,np.zeros(3)))

array([0.106, 0.059, 0.17 , 0.084, 0.09 , 0.052, 0.153, 0.159, 0.049,
       0.086, 0.055, 0.08 , 0.052, 0.098, 0.038, 0.059, 0.177, 0.047,
       0.046, 0.205, 0.09 , 0.127, 0.181, 0.171, 0.106, 0.193, 0.059,
       0.082, 0.205, 0.079, 0.066, 0.101, 0.061, 0.126, 0.165, 0.075,
       0.165, 0.199, 0.056, 0.072, 0.228, 0.   , 0.   , 0.   ])

In [59]:
train_X_44 = list()
for i in (range(len(train_X))):
  row = np.array(train_X.iloc[i])
  row = np.concatenate((row,np.zeros(3)))
  train_X_44.append(row)
np.array(train_X_44).shape

(10452, 44)

In [60]:
test_X_44 = list()
for i in (range(len(test_X))):
  row = np.array(test_X.iloc[i])
  row = np.concatenate((row,np.zeros(3)))
  test_X_44.append(row)
np.array(test_X_44).shape

(258829, 44)

In [61]:
train_y = np.array(train_Y)
test_y = np.array(test_Y)
train_x = list()
train_x.append(train_X_44)
train_x = np.dstack(train_x)
test_x = list()
test_x.append(test_X_44)
test_x = np.dstack(test_x)
print('Shape of the train features tensor',train_x.shape)
print('Shape of the train labels tensor',train_y.shape)
print('Shape of the test features tensor',test_x.shape)
print('Shape of the test labels tensor',test_y.shape)

Shape of the train features tensor (10452, 44, 1)
Shape of the train labels tensor (10452, 1)
Shape of the test features tensor (258829, 44, 1)
Shape of the test labels tensor (258829,)


In [64]:
n_steps, n_length, n_features = 4, 11, 1
train_x = train_x.reshape((train_x.shape[0], n_steps, n_length, n_features))
test_x = test_x.reshape((test_x.shape[0], n_steps, n_length, n_features))
print('Shape of the train features tensor',train_x.shape)
print('Shape of the test features tensor',test_x.shape)

Shape of the train features tensor (10452, 4, 11, 1)
Shape of the test features tensor (258829, 4, 11, 1)


In [65]:
# zero-offset class values
train_y = train_y - 1
test_y = test_y - 1
# one hot encode y
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [66]:
print('Shape of the test labels tensor',test_y.shape)
print('Shape of the train labels tensor',train_y.shape)

Shape of the test labels tensor (258829, 2)
Shape of the train labels tensor (10452, 2)


In [77]:
n_timesteps, n_features, n_outputs = train_x.shape[2], train_x.shape[3], train_y.shape[1]
print(n_timesteps,n_features,n_outputs)

11 1 2


In [ ]:
from keras.layers import TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten

In [103]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_32 (TimeDis (None, None, 9, 64)       256       
_________________________________________________________________
time_distributed_33 (TimeDis (None, None, 7, 64)       12352     
_________________________________________________________________
time_distributed_34 (TimeDis (None, None, 7, 64)       0         
_________________________________________________________________
time_distributed_35 (TimeDis (None, None, 3, 64)       0         
_________________________________________________________________
time_distributed_36 (TimeDis (None, None, 192)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               117200    
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)              

In [104]:
verbose, epochs, batch_size = 0, 15, 64
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

In [105]:
# evaluate model
_, accuracy = model.evaluate(train_x, train_y, batch_size=batch_size, verbose=0)
print(accuracy)

0.7154611349105835


In [106]:
# evaluate model on test set
_, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
print(accuracy)

0.6187559962272644


*     Epochs 25, Train:71 Test:77
*     Epochs 50 Train:84 Test:67
*     Epochs 10 Train 70 Test 68
*     Epochs 15 Train 71 Test 61



---
## Evaluate without Padding

